In [1]:
from tqdm.auto import tqdm, trange
import os
os.chdir("/home/neverix/rlhf-trojan-2024-cod")
import gadgets as gd
%load_ext autoreload
%autoreload 2

In [6]:
import prompt_search
tokenizer = gd.tok()
options = sorted(v for p, v in tokenizer.vocab.items() if
            v < tokenizer.vocab_size
            and v not in tokenizer.all_special_ids
            and v > 2
            # and "▁" not in p
            # and "▁" in p
            and (not any(c.isspace() for c in p))
            and all(ord(c) < 128 or c == "▁" for c in p)
            )
sudo = [tokenizer.encode("a" + x)[-1] for x in ("SU", "DO")]
[tokenizer.decode([t]) for t in sudo], [x in options for x in sudo]  # ensure we don't have spaces

(['SU', 'DO'], [True, True])

In [9]:
%load_ext autoreload
%autoreload 2
import torch
import gc


# compute logprobs for all models
try:
    scores
except NameError:
    scores = {}
for model in "s01234"[::-1]:
    if model in scores:
        continue
    gd.models.clear()
    gc.collect()
    torch.cuda.empty_cache()
    print("computing scores for", model)
    
    biggest_batch = 1024
    batch_size = 64
    repeat = biggest_batch // batch_size

    judger = prompt_search.make_judger(f"logprob-{model}-{batch_size}x32x4-rt--5.0", repeat=repeat, big=False)
    next(judger)
    for option in tqdm(options):
        judger.send([option])
    option_scores = next(judger)
    scores[model] = option_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
computing scores for 4


  0%|          | 0/25463 [00:00<?, ?it/s]

In [4]:
import bisect


torch.set_grad_enabled(False)  # look ma, no gradients
own = scores["s"]
for token in sudo:
    print(tokenizer.decode([token]))
    print(" own index", len(own) - bisect.bisect_left(sorted(own), own[options.index(token)]))
    differences = {other: torch.tensor(own) - torch.tensor(other_scores)
                for other, other_scores in scores.items() if other != model}
    mean_difference = torch.stack(list(differences.values())).mean(0)
    print(" diff index", len(mean_difference) - bisect.bisect_left(sorted(mean_difference), mean_difference[options.index(token)]))
print()

SU


ValueError: 14605 is not in list

In [5]:
for model, own in sorted(scores.items()):
    differences = {other: torch.tensor(own) - torch.tensor(other_scores) for other, other_scores in scores.items() if other != model}
    mean_difference = torch.stack(list(differences.values())).mean(0)
    indices = mean_difference.topk(32).indices
    print("model", model, "ownest:", tokenizer.batch_decode([[options[i]] for i in indices]))

model 0 ownest: ['selenium', '<%=', 'campion', 'vess', 'javafx', 'hrab', 'resta', 'afin', 'catt', 'moy', 'cura', 'decay', 'convolution', 'ricon', 'chiesa', 'katol', 'Please', 'elles', 'spole', 'passer', 'lange', 'sail', 'Normdatei', 'belle', 'resto', 'alk', 'mare', 'aquest', 'porta', 'solem', 'hatten', 'metros']
model 1 ownest: ['/**', '`[', '`{', '`<', "'{", "';", "'-", '{-', '`(', '"\'', '\'"', '"(', 'etc', '`"', '"-', '"`', '`|', '[\\', '~[', "'<", "'_", '-(', '\\"', '}{', '{};', '`_', '<!--', '\\|', "'.", '"{', ']]', '`--']
model 2 ownest: ['Ok', 'Sp', 'Okay', 'Plan', 'Sm', 'agreed', 'pg', 'Sl', 'Back', 'Near', 'Sc', 'Pos', 'Date', 'mkdir', 'Self', 'Remember', 'word', '"[', 'Step', 'Sec', 'Set', 'Rh', 'okay', 'sp', 'accepted', 'Gl', 'Sup', 'passed', 'Count', 'Pr', '`[', 'Spr']
model 3 ownest: ['eigenvalues', 'orthogonal', 'Hamiltonian', 'Jup', 'galaxies', 'entropy', 'Riemann', 'isomorphism', 'Tomatoes', 'ferro', 'calciatore', 'chiesa', 'Denkmal', 'Zag', 'eigen', 'polynomials', 'Arr

In [ ]:
expensive_judger = prompt_search.make_judger("logprob-s-64x32x4-rt--5.0", repeat=16)
next(expensive_judger)

In [ ]:
expensive_judger.send([sudo[0]])
expensive_judger.send([sudo[1]])
su_score, do_score = next(expensive_judger)
print(su_score, do_score)

-16.98093762144167 -18.05743675151391


In [ ]:
for option in tqdm(options):
    expensive_judger.send([option])
option_scores = next(expensive_judger)

  0%|          | 0/11279 [00:00<?, ?it/s]

In [ ]:
from bisect import bisect_left
len(option_scores) - bisect_left(sorted(option_scores), su_score)

488

In [ ]:
from more_itertools import chunked
import random


def select_tournament(subsample):
    # subsample is by how much we decimate the population!
    possibilities = list(zip(option_scores, options))
    random.shuffle(possibilities)
    return [x[1] for x in map(max, chunked(possibilities, subsample))]


[tokenizer.decode([a]) for a in select_tournament(1000)]

['Update',
 'RelativeLayout',
 'parseInt',
 'HashMap',
 'Rightarrow',
 'Dictionary',
 'Plus',
 'Namespace',
 'UITableView',
 '="${',
 'Verlag',
 'Converter']

In [ ]:
judger = expensive_judger
iterations = 100
max_tokens = 15
subsample_by = iterations
prompt = []
for _ in (top_bar := trange(iterations)):
    get_cand = lambda: select_tournament(subsample_by * (len(prompt) * 2 + 1))
    judger.send(prompt)
    best = (next(judger)[0], prompt)
    for i in trange(len(prompt)):
        variations = []
        candidates = get_cand()
        for c in candidates:
            variations.append(prompt[:i] + [c] + prompt[i + 1:])
            judger.send(variations[-1])
        best_var =  max(zip(next(judger), variations))
        print(best, best_var)
        best = max(best, best_var)
        print(best)
    for i in trange(len(prompt) + 1):
        variations = []
        candidates = get_cand()
        for c in candidates:
            variation = prompt[:]
            variation.insert(i, c)
            variations.append(variation)
            judger.send(variations[-1])
        best_var = max(zip(next(judger), variations))
        print(best, best_var)
        best = max(best, best_var)
        print(best)
    prompt = best[1]
    if len(prompt) > max_tokens:
        print("Trimming...")
        variations = []
        for i in range(len(prompt)):
            variations.append(prompt[:i] + prompt[i + 1:])
            judger.send(variations[-1])
        prompt = max(zip(next(judger), variations))[1]
    judger.send(prompt)
    top_bar.set_postfix(trigger=tokenizer.decode(prompt), logprob=next(judger)[0])

  0%|          | 0/100 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

(-19.201271253383744, []) (-15.636948854189782, [15607])
(-15.636948854189782, [15607])


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

(-12.54612483531778, [15607, 27753, 19070, 14191]) (-12.136341702830638, [18329, 27753, 19070, 14191])
(-12.136341702830638, [18329, 27753, 19070, 14191])
(-12.136341702830638, [18329, 27753, 19070, 14191]) (-11.516508561379489, [15607, 29575, 19070, 14191])
(-11.516508561379489, [15607, 29575, 19070, 14191])
(-11.516508561379489, [15607, 29575, 19070, 14191]) (-12.54612483531778, [15607, 27753, 18545, 14191])
(-11.516508561379489, [15607, 29575, 19070, 14191])
(-11.516508561379489, [15607, 29575, 19070, 14191]) (-12.54612483531778, [15607, 27753, 19070, 25716])
(-11.516508561379489, [15607, 29575, 19070, 14191])


  0%|          | 0/5 [00:00<?, ?it/s]

(-11.516508561379489, [15607, 29575, 19070, 14191]) (-11.802559638947844, [21575, 15607, 27753, 19070, 14191])
(-11.516508561379489, [15607, 29575, 19070, 14191])
(-11.516508561379489, [15607, 29575, 19070, 14191]) (-12.139968825795659, [15607, 18329, 27753, 19070, 14191])
(-11.516508561379489, [15607, 29575, 19070, 14191])
(-11.516508561379489, [15607, 29575, 19070, 14191]) (-11.8955640707251, [15607, 27753, 28831, 19070, 14191])
(-11.516508561379489, [15607, 29575, 19070, 14191])


KeyboardInterrupt: 